# Routing Assignment

Congratulations! You just convinced a group of investors to put money into your startup competing against _Instacart_ for delivering specialty pet supplies for hamster grooming. Immediately upon publishing your website, the City of Dallas signs up to have a pallet of hamster grooming supplies delivered to "1500 Marilla Street, Dallas, TX" (Dallas City Hall--where all office animals are banned except hamsters). In addition, you received an order from Annamelissa Uhumehlemehe (a well-known childish hamster lady) who lives at "11322 Cactus Lane, Dallas, TX". You notice other addresses rolling in--_all in the City of Dallas_ (none in Garland, Grand Prarie, Ft. Worth, etc.). So you better hurry because you don't want to lose out on sales!

You immediately go to the https://openstreetmap.org website and verify that these addresses are good addresses in OpenStreetMap including your warehouse at "5351 Fults Blvd., Dallas, TX" (just off Samuell Blvd., and South Buckner). One of your investors' requirements is that you not spend money on Google products or any other product. _You must use OpenStreetMap_ for your routing application.

Your project is to write an application that optimizes deliveries to 20 addresses at a time in the City of Dallas using OpenStreetMap's API. You may pick 18 other addresses at random but, be careful to validate them in OpenStreetMap. Here are some address details:

- Starting (warehouse) address is "5351 Fults Blvd., Dallas, TX"
- You must include "1500 Marilla Street, Dallas, TX", and "11322 Cactus Lane, Dallas, TX" in your list of delivery addresses
- You may pick 18 other addresses but they also have to process any verified list of OpenStreetMap-valid addresses

Your application must:

- Optimize the route starting from your warehouse address with the shortest distance to complete the route
- Display the destinations and the distance between them (refer to the example in the assignment on Canvas)
- Display a map with the most efficient route displayed (refer to the example in the assignment on Canvas)
- Accept a list of up to 20 delivery addresses in Dallas and generate a new route list and map
- Oh wait! The example on Canvas shows distances in km! We use **miles** here so don't forget to convert!

The rubric for this assignment includes:

- All required components are present and your program works as per the specifications (above)
- Additional innovations may be (should be) added to differentiate your app from your classmates
- Your program is well-commented and demonstrates that you know how it works
- Any and all functions should include _docstrings_ that appear when the "help( )" function is called
- The class presentation is excellent

The cells below instantiate packages that may be of use as a starting place. You may alter them but remember: _you must use OpenStreetMap_ to complete this assignment.

Advanced Features

1. Intelligent Traffic Prediction:

- Time-based traffic multipliers
- Location-based congestion factors
- Real-world rush hour considerations


2. Multi-Constraint Optimization:

- Priority-based routing
- Time window constraints
- Vehicle capacity limits
- Package handling times
- Electric vehicle energy consumption


3. Advanced Genetic Algorithm:

- Population-based optimization
- Intelligent crossover and mutation
- Multi-factor fitness function
- Parallel processing capability


4. Enhanced Visualization:

- Priority-based color coding
- Traffic intensity route coloring
- Detailed stop information
- Interactive legend
- Real-time traffic indicators


5. Business Intelligence Features:

- Delivery time windows
- Priority-based scheduling
- Package size optimization
- Vehicle capacity management
- Energy consumption tracking

In [1]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
import folium
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Tuple, Dict
from dataclasses import dataclass
import random
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

In [2]:


@dataclass
class DeliveryStop:
    """Data class for delivery stop information"""
    address: str
    coordinates: Tuple[float, float]
    priority: int = 1  # 1-5, 5 being highest
    time_window: Tuple[int, int] = (9, 17)  # 24-hour format
    package_size: float = 1.0  # cubic feet
    handling_time: int = 5  # minutes

class IntelligentDeliveryOptimizer:
    """
    Advanced delivery route optimizer with unique features:
    - Genetic algorithm for route optimization
    - Time window constraints
    - Priority-based routing
    - Traffic pattern prediction
    - Vehicle capacity optimization
    - Battery-friendly routing for electric vehicles
    - Multi-threaded route calculation
    """
    
    def __init__(self):
        """Initialize the optimizer with advanced features"""
        self.geolocator = Nominatim(user_agent="hamster_delivery_optimizer")
        print("Loading Dallas infrastructure data...")
        self.dallas_graph = ox.graph_from_place('Dallas, Texas, USA', network_type='drive')
        self.dallas_graph_proj = ox.project_graph(self.dallas_graph)
        
        # Traffic patterns by hour (mock data - could be replaced with real historical data)
        self.traffic_patterns = {
            'morning_rush': {h: 1.5 for h in range(7, 10)},
            'evening_rush': {h: 1.6 for h in range(16, 19)},
            'normal': {h: 1.0 for h in range(24)}
        }
        
        # Initialize genetic algorithm parameters
        self.population_size = 100
        self.generations = 50
        self.mutation_rate = 0.1
        
        # Vehicle constraints
        self.max_capacity = 100  # cubic feet
        self.max_battery_range = 200  # miles
        
    def predict_traffic_multiplier(self, hour: int, location: Tuple[float, float]) -> float:
        """Predict traffic intensity based on time and location"""
        base_multiplier = self.traffic_patterns['normal'].get(hour, 1.0)
        if hour in self.traffic_patterns['morning_rush']:
            base_multiplier = self.traffic_patterns['morning_rush'][hour]
        elif hour in self.traffic_patterns['evening_rush']:
            base_multiplier = self.traffic_patterns['evening_rush'][hour]
            
        # Add location-based adjustment (mock implementation)
        downtown_dallas = (32.7767, -96.7970)
        distance_to_downtown = np.sqrt(
            (location[0] - downtown_dallas[0])**2 + 
            (location[1] - downtown_dallas[1])**2
        )
        location_multiplier = 1 + (0.2 * (1 - min(distance_to_downtown * 10, 1)))
        
        return base_multiplier * location_multiplier
        
    def calculate_energy_consumption(self, route: List[DeliveryStop]) -> float:
        """Calculate estimated energy consumption for electric vehicles"""
        total_energy = 0
        for i in range(len(route) - 1):
            distance = self.calculate_distance(route[i].coordinates, route[i+1].coordinates)
            # Simple energy model: distance * elevation_factor * traffic_factor
            elevation_factor = 1.0  # Could be enhanced with actual elevation data
            current_hour = datetime.now().hour
            traffic_factor = self.predict_traffic_multiplier(current_hour, route[i].coordinates)
            total_energy += distance * elevation_factor * traffic_factor
        return total_energy
        
    def fitness_function(self, route: List[DeliveryStop]) -> float:
        """
        Calculate route fitness considering multiple factors:
        - Total distance
        - Priority adherence
        - Time windows
        - Traffic conditions
        - Energy consumption
        """
        total_distance = 0
        total_time = 0
        current_load = 0
        current_time = 8 * 60  # Start at 8:00 AM (in minutes)
        penalty = 0
        
        for i in range(len(route) - 1):
            # Distance and time calculations
            distance = self.calculate_distance(route[i].coordinates, route[i+1].coordinates)
            traffic_mult = self.predict_traffic_multiplier(current_time // 60, route[i].coordinates)
            travel_time = (distance * traffic_mult * 60) / 30  # Assuming 30mph average
            
            # Update metrics
            total_distance += distance
            total_time += travel_time + route[i].handling_time
            current_time += travel_time + route[i].handling_time
            current_load += route[i].package_size
            
            # Apply penalties
            if current_load > self.max_capacity:
                penalty += 1000
            
            # Time window penalties
            delivery_hour = current_time // 60
            if delivery_hour < route[i].time_window[0] or delivery_hour > route[i].time_window[1]:
                penalty += 500
                
            # Priority penalties
            if route[i].priority > 3 and i > len(route) // 2:
                penalty += 200 * route[i].priority
                
        # Energy consumption penalty
        energy_consumption = self.calculate_energy_consumption(route)
        if energy_consumption > self.max_battery_range:
            penalty += 1000
            
        return total_distance + penalty
        
    def genetic_algorithm(self, stops: List[DeliveryStop]) -> List[DeliveryStop]:
        """Implement genetic algorithm for route optimization"""
        def create_population():
            return [random.sample(stops[1:-1], len(stops)-2) for _ in range(self.population_size)]
            
        def crossover(parent1, parent2):
            point = random.randint(0, len(parent1)-1)
            child = parent1[:point]
            for stop in parent2:
                if stop not in child:
                    child.append(stop)
            return child
            
        def mutate(route):
            if random.random() < self.mutation_rate:
                i, j = random.sample(range(len(route)), 2)
                route[i], route[j] = route[j], route[i]
            return route
            
        # Initialize population
        population = create_population()
        
        # Evolution process
        for generation in range(self.generations):
            # Add warehouse to start and end
            full_routes = [[stops[0]] + route + [stops[-1]] for route in population]
            
            # Calculate fitness
            fitness_scores = [self.fitness_function(route) for route in full_routes]
            
            # Select parents
            parents = [population[i] for i in np.argsort(fitness_scores)[:self.population_size//2]]
            
            # Create new population
            new_population = []
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(parents, 2)
                child = crossover(parent1, parent2)
                child = mutate(child)
                new_population.append(child)
                
            population = new_population
            
        # Return best route
        best_route_idx = np.argmin([self.fitness_function([stops[0]] + route + [stops[-1]]) 
                                  for route in population])
        return [stops[0]] + population[best_route_idx] + [stops[-1]]
        
    def create_visualization(self, route: List[DeliveryStop]) -> None:
        """Create an enhanced visualization of the route"""
        m = folium.Map(location=[32.7767, -96.7970], zoom_start=12)
        
        # Create a color gradient based on stop priority
        def get_color(priority):
            colors = ['blue', 'green', 'yellow', 'orange', 'red']
            return colors[priority-1]
            
        # Add markers with enhanced information
        for i, stop in enumerate(route[:-1]):  # Skip last duplicate warehouse
            color = 'purple' if i == 0 else get_color(stop.priority)
            popup_html = f"""
                <b>Stop {i}</b><br>
                Address: {stop.address}<br>
                Priority: {stop.priority}<br>
                Time Window: {stop.time_window[0]}:00-{stop.time_window[1]}:00<br>
                Package Size: {stop.package_size} cu.ft<br>
                Handling Time: {stop.handling_time} min
            """
            
            folium.Marker(
                stop.coordinates,
                popup=folium.Popup(popup_html, max_width=300),
                icon=folium.Icon(color=color)
            ).add_to(m)
        
        # Add route lines with traffic intensity coloring
        for i in range(len(route) - 1):
            current_hour = datetime.now().hour
            traffic_mult = self.predict_traffic_multiplier(current_hour, route[i].coordinates)
            
            # Color based on traffic intensity
            color = 'green' if traffic_mult <= 1.2 else 'yellow' if traffic_mult <= 1.5 else 'red'
            
            # Get detailed path between points
            path = self.get_detailed_path(route[i].coordinates, route[i+1].coordinates)
            
            folium.PolyLine(
                path,
                weight=2,
                color=color,
                opacity=0.8,
                popup=f'Traffic Multiplier: {traffic_mult:.2f}'
            ).add_to(m)
            
        # Add legend
        legend_html = """
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; 
                    padding: 10px; border: 2px solid grey; border-radius: 5px">
            <p><b>Legend</b></p>
            <p><i class="fa fa-circle" style="color:purple"></i> Warehouse</p>
            <p><i class="fa fa-circle" style="color:red"></i> High Priority</p>
            <p><i class="fa fa-circle" style="color:orange"></i> Medium-High Priority</p>
            <p><i class="fa fa-circle" style="color:yellow"></i> Medium Priority</p>
            <p><i class="fa fa-circle" style="color:green"></i> Low Priority</p>
            <p><i class="fa fa-circle" style="color:blue"></i> Lowest Priority</p>
            <br>
            <p><b>Traffic Conditions:</b></p>
            <p><i class="fa fa-line" style="color:green"></i> Light Traffic</p>
            <p><i class="fa fa-line" style="color:yellow"></i> Moderate Traffic</p>
            <p><i class="fa fa-line" style="color:red"></i> Heavy Traffic</p>
        </div>
        """
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # Save map
        m.save('optimized_route_advanced.html')
        
    def get_detailed_path(self, start: Tuple[float, float], end: Tuple[float, float]) -> List[Tuple[float, float]]:
        """Get detailed path coordinates between two points"""
        start_node = ox.nearest_nodes(self.dallas_graph, start[1], start[0])
        end_node = ox.nearest_nodes(self.dallas_graph, end[1], end[0])
        
        try:
            route = nx.shortest_path(self.dallas_graph, start_node, end_node, weight='length')
            return [(self.dallas_graph.nodes[node]['y'], self.dallas_graph.nodes[node]['x']) 
                   for node in route]
        except nx.NetworkXNoPath:
            return [start, end]
            
    def calculate_distance(self, start: Tuple[float, float], end: Tuple[float, float]) -> float:
        """Calculate distance between two points in miles"""
        try:
            start_node = ox.nearest_nodes(self.dallas_graph_proj, start[1], start[0])
            end_node = ox.nearest_nodes(self.dallas_graph_proj, end[1], end[0])
            route = nx.shortest_path(self.dallas_graph_proj, start_node, end_node, weight='length')
            return sum(ox.utils_graph.get_route_edge_attributes(
                self.dallas_graph_proj, route, 'length')) * 0.000621371
        except nx.NetworkXNoPath:
            return float('inf')

def create_sample_delivery_stops():
    """Create sample delivery stops with realistic constraints"""
    warehouse = DeliveryStop(
        "5351 Fults Blvd., Dallas, TX",
        (32.7767, -96.7970),
        priority=1,
        time_window=(8, 18),
        package_size=0,
        handling_time=15
    )
    
    city_hall = DeliveryStop(
        "1500 Marilla Street, Dallas, TX",
        (32.7767, -96.7970),
        priority=5,  # High priority
        time_window=(9, 16),
        package_size=2.5,
        handling_time=10
    )
    
    hamster_lady = DeliveryStop(
        "11322 Cactus Lane, Dallas, TX",
        (32.8915, -96.7170),
        priority=4,
        time_window=(10, 14),  # Limited time window
        package_size=1.5,
        handling_time=8
    )
    
    # Add more stops here...
    
    return [warehouse, city_hall, hamster_lady, warehouse]  # Return to warehouse

In [3]:
def main():
    """Main function to demonstrate the advanced routing optimizer"""
    print("Initializing Advanced Delivery Optimizer...")
    optimizer = IntelligentDeliveryOptimizer()
    
    # Create sample delivery stops with varying priorities and constraints
    delivery_stops = [
        # Warehouse (start and end point)
        DeliveryStop(
            "5351 Fults Blvd., Dallas, TX",
            optimizer.geolocator.geocode("5351 Fults Blvd., Dallas, TX").point[:2],
            priority=1,
            time_window=(8, 18),
            package_size=0,
            handling_time=15
        ),
        
        # City Hall (high priority)
        DeliveryStop(
            "1500 Marilla Street, Dallas, TX",
            optimizer.geolocator.geocode("1500 Marilla Street, Dallas, TX").point[:2],
            priority=5,
            time_window=(9, 16),
            package_size=2.5,
            handling_time=10
        ),
        
        # Annamelissa's address (high priority)
        DeliveryStop(
            "11322 Cactus Lane, Dallas, TX",
            optimizer.geolocator.geocode("11322 Cactus Lane, Dallas, TX").point[:2],
            priority=4,
            time_window=(10, 14),
            package_size=1.5,
            handling_time=8
        ),
        
        # Additional Dallas addresses with varied priorities
        DeliveryStop(
            "8687 N Central Expy, Dallas, TX",
            optimizer.geolocator.geocode("8687 N Central Expy, Dallas, TX").point[:2],
            priority=3,
            time_window=(9, 17),
            package_size=1.8,
            handling_time=7
        ),
        
        DeliveryStop(
            "2909 McKinney Ave, Dallas, TX",
            optimizer.geolocator.geocode("2909 McKinney Ave, Dallas, TX").point[:2],
            priority=2,
            time_window=(11, 15),
            package_size=1.2,
            handling_time=5
        ),
        
        # Add warehouse again as final destination
        DeliveryStop(
            "5351 Fults Blvd., Dallas, TX",
            optimizer.geolocator.geocode("5351 Fults Blvd., Dallas, TX").point[:2],
            priority=1,
            time_window=(8, 18),
            package_size=0,
            handling_time=15
        )
    ]
    
    # Run the optimization
    try:
        print("\nCalculating optimal route...")
        optimized_route = optimizer.genetic_algorithm(delivery_stops)
        
        # Generate detailed route statistics
        print("\nGenerating route statistics...")
        total_distance = 0
        total_time = 0
        current_time = datetime.now().replace(hour=8, minute=0)  # Start at 8 AM
        
        print("\nOptimized Route Details:")
        print("------------------------")
        for i in range(len(optimized_route) - 1):
            distance = optimizer.calculate_distance(
                optimized_route[i].coordinates,
                optimized_route[i + 1].coordinates
            )
            
            # Calculate travel time with traffic
            traffic_mult = optimizer.predict_traffic_multiplier(
                current_time.hour,
                optimized_route[i].coordinates
            )
            travel_time = (distance * traffic_mult * 60) / 30  # minutes
            
            total_distance += distance
            total_time += travel_time + optimized_route[i].handling_time
            
            print(f"\nStop {i + 1}: {optimized_route[i].address}")
            print(f"Priority Level: {optimized_route[i].priority}")
            print(f"Time Window: {optimized_route[i].time_window[0]}:00-{optimized_route[i].time_window[1]}:00")
            print(f"Expected Arrival: {current_time.strftime('%I:%M %p')}")
            print(f"Distance to next stop: {distance:.2f} miles")
            print(f"Traffic Multiplier: {traffic_mult:.2f}x")
            
            current_time += timedelta(minutes=travel_time + optimized_route[i].handling_time)
        
        print("\nRoute Summary:")
        print(f"Total Distance: {total_distance:.2f} miles")
        print(f"Total Estimated Time: {total_time/60:.1f} hours")
        print(f"Number of Stops: {len(optimized_route)-1}")  # Excluding return to warehouse
        
        # Create visualization
        print("\nGenerating route visualization...")
        optimizer.create_visualization(optimized_route)
        print("\nRoute map has been saved as 'optimized_route_advanced.html'")
        
        # Calculate energy consumption
        energy = optimizer.calculate_energy_consumption(optimized_route)
        print(f"\nEstimated Energy Consumption: {energy:.1f} kWh")
        if energy > optimizer.max_battery_range:
            print("WARNING: Route may exceed vehicle battery range!")
            
    except Exception as e:
        print(f"\nError during optimization: {str(e)}")
        raise

In [4]:
main()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x11177a580>>
Traceback (most recent call last):
  File "/Users/ephraim888sun/.pyenv/versions/3.9.6/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
